In [1]:
import os
import glob
import cv2
from pathlib import Path
import numpy as np
from torch.nn.functional import threshold


In [ ]:
def imshow_components(labels):
    # Map component labels to hue val
    label_hue = np.uint8(179 * labels / np.max(labels))
    blank_ch = 255 * np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue == 0] = 0

    return labeled_img

In [ ]:
for img in predictions_masks_img:
    filename = Path(img).stem
    img = cv2.imread(img, 0)
    img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]  # ensure binary
    num_labels, labels_im = cv2.connectedComponents(img)

    print(num_labels)

    #labeled_img = imshow_components(labels_im)

In [ ]:
predictions_masks_path = "test_dataset/predictions/sor"
os.makedirs("test_dataset/labeled_masks", exist_ok=True)

predictions_masks_img = sorted(glob.glob(predictions_masks_path + "/*.png"))

In [ ]:
from PIL import Image

In [ ]:
import numpy as np
#for img in predictions_masks_img:
img = predictions_masks_img[0]
filename = Path(img).stem
img = cv2.imread(img, 0)
img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]  # ensure binary
cv2.imshow(img)
num_labels, labels_im = cv2.connectedComponents(img)img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]  # ensure binary
cv2.imshow(img)
num_labels, labels_im = cv2.connectedComponents(img)
print(np.unique(labels_im))
print(num_labels)
labeled_img = Image.fromarray(labels_im)


In [2]:
import cv2

In [51]:
predictions_masks_path = "test_dataset/predictions/sor"
masks_path = "test_dataset/masks"

predictions_masks_img = sorted(glob.glob(predictions_masks_path + "/*.png"))
masks_img = sorted(glob.glob(masks_path + "/*.tif"))


imkey = 0

predicted = cv2.imread(predictions_masks_img[imkey], 0)
mask= cv2.imread(masks_img[imkey], 0)


predicted = cv2.threshold(predicted, 127, 255, cv2.THRESH_BINARY)[1]  # ensure binary
mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)[1]  # ensure binary

num_labels_predicted, labels_im_predicted = cv2.connectedComponents(predicted)
num_labels_mask, labels_im_mask = cv2.connectedComponents(mask)

iou_matrix = np.zeros((num_labels_predicted, num_labels_mask))

fp = 0
threshold = 0
for i in range(1, num_labels_predicted):
    for j in range(1, num_labels_mask):
        intersection = np.logical_and(labels_im_predicted == i, labels_im_mask == j).sum()
        union = np.logical_or(labels_im_predicted == i, labels_im_mask == j).sum()
        iou_matrix[i, j] = intersection / union

        if i == j and iou_matrix[i, j] <= threshold:
            fp+=fp

        if i > j and j > num_labels_predicted < num_labels_mask and i > num_labels_mask-1 and iou_matrix[i, j] <= threshold:
            fp+=fp

        if i < j and j > num_labels_predicted > num_labels_mask and j > num_labels_predicted-1 and iou_matrix[i, j] <= threshold:
            fp+=fp

#lignes
tp_array = (iou_matrix > threshold).sum(axis=1)

#colonnes
fn_array = (iou_matrix > threshold).sum(axis=0)

#diagonale
#fp = np.sum(np.diag(iou_matrix) <= threshold)

tp = (tp_array == 0).sum()
fn = (fn_array == 0).sum()


f1score = (2*tp)/(2*tp+fp+fn)

f1score

np.float64(0.25)

In [52]:
tp, fp, fn


(np.int64(1), 0, np.int64(6))